In [3]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.firefox.options import Options


In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [3]:
#options = Options()
#options.add_argument("--headless")
#options.set_headless(headless=True)
Firefox = webdriver.Firefox(executable_path = "C:\Program Files\Mozilla Firefox\geckodriver.exe")


In [4]:
#из примера. превращаем в xlsx file
ff = [
1000000000,
7736207543,
23,
1661056318,
111111111111111,
5752030868,
6732161906,
]
gg = pd.DataFrame(ff, columns=['inn'])
gg.to_excel('gg.xlsx', index=False)
gg = pd.read_excel('gg.xlsx')
gg

,inn
0,1000000000
1,7736207543
2,23
3,1661056318
4,111111111111111
5,5752030868
6,6732161906


In [7]:
def pars(list):
    Firefox.get('https://egrul.nalog.ru/')
    time.sleep (0.1)
    do = Firefox.find_element_by_css_selector('#query')
    display('count')
    do.click()
    time.sleep (0.005)
    for i in list:
        do.send_keys(i)
        time.sleep (0.005)
    time.sleep (0.1)    
    WebDriverWait(Firefox, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#btnSearch'))).click()
    time.sleep (2)
    if (Firefox.find_element_by_css_selector('div.pane:nth-child(1) > div:nth-child(1) > p:nth-child(1)').text != 'По заданным критериям поиска данных не найдено.'):
        time.sleep (2)
        display('download')
        #    Firefox.execute_script("arguments[0].click();", Firefox.find_element_by_css_selector('button.btn-with-icon:nth-child(2)'))
        WebDriverWait(Firefox, 20).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div[3]/div/div[1]/div[4]/div/div[3]/button'))).click()
        i=0
        for i in range (20):
            do.send_keys(Keys.BACK_SPACE)
            time.sleep (0.1)
            i+=1
    else:
        print('отсутствует в базе')
        pass

In [8]:
%%time

list = []
for i in gg['inn']:
    if (len(str(i))) == 10:  # корректные ИНН если 10 значные
        for ii in range (9, -1, -1):
            z = i//10**ii
            i = i-z*10**(ii)
            list.append(z)
        display(list)
        
     
        pars(list)
    
        list = []

    


[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

'count'

отсутствует в базе


[7, 7, 3, 6, 2, 0, 7, 5, 4, 3]

'count'

...

'download'

[6, 7, 3, 2, 1, 6, 1, 9, 0, 6]

'count'

'download'

Wall time: 53 s


## Tabula

In [116]:
import tabula
#tabula.environment_info()

In [117]:
# экспорт pdf в xlsx

files = []
import os
for file in os.listdir('.'):
    if file.endswith(".pdf"):
        files = (os.path.join('', file))
        display(files)
        dfs = tabula.read_pdf(files, encoding='utf-8', stream=True, pages="all")
        pd.DataFrame(dfs).to_excel(str.split(files, sep = '.')[0] + '.xlsx')
        display('end')

'ul-1027700229193-20201001123319.pdf'

'end'

'ul-1035752001306-20201001123210.pdf'

'end'

'ul-1181690033315-20201001123159.pdf'

'end'

'ul-1186733007230-20201001123221.pdf'

'end'

### Camelot


In [5]:
%%time

import camelot
#выгрузка camelot to xlsx

dfs = {}
files = []
import os
for file in os.listdir('.'):
    if file.endswith(".pdf"):
        files = (os.path.join('', file))
        display(files)
        dfs[file] = camelot.read_pdf(files, flavor='stream', pages = 'all')
        pd.DataFrame(dfs[file][0].df).to_excel(str.split(files, sep = '.')[0] + '_from_camelot' + '.xlsx')
  #      display('end')

'ul-1027700229193-20201001123319.pdf'

'ul-1035752001306-20201001123210.pdf'

'ul-1181690033315-20201001123159.pdf'

'ul-1186733007230-20201001123221.pdf'

Wall time: 18.6 s


In [42]:
#cоздали перечень уникальных значений
dfs_all        = pd.DataFrame()
dfs_all_concat = pd.DataFrame()
for z in dfs.keys():
    for n in range(30):
        try:
            #display(n)
            df_n_z = dfs[z][n].df
        #    df_n_z['inn'] = 
            dfs_all = dfs_all.append(df_n_z)
        except:
            pass
    dfs_all_concat = pd.concat([dfs_all_concat, dfs_all], axis=0)
    dfs_all        = pd.DataFrame()
dfs_all_concat        = pd.DataFrame(dfs_all_concat[1].unique(), columns = [1])
#dfs_all_concat.colum = ['merg']
dfs_all_concat   

#merg 

dfs_all        = pd.DataFrame()
for z in dfs.keys():
    for n in range(2):  
        try:
            #display(n)
            df_n_z = dfs[z][n].df
        #    df_n_z['inn'] = 
            dfs_all = dfs_all.append(df_n_z)
        except:
            pass
    dfs_all_concat = pd.merge(dfs_all_concat, dfs_all, on = [1], how = 'right')
    dfs_all        = pd.DataFrame()

In [44]:
dfs_all_concat[dfs_all_concat[1]. isin(['Сокращенное наименование', 'Улица (проспект, переулок и т.д.)', \
'Дом (владение и т.п.)', 'Почтовый индекс', 'Субъект Российской Федерации', 'КПП', 'Вид'])]


,1,0_x,2_x,0_y,2_y,0_x,2_x,0_y,2_y
20404,Сокращенное наименование,2,"ООО ""ЯНДЕКС""",2,"ООО ""ЧОП ""ШТОРМ""",2,"ООО ""УНИСТРОЙ""",2,"ООО ""СТК"""
36606,Почтовый индекс,4,119021,4,302042,4,420127,4,214032
36607,Субъект Российской Федерации,5,ГОРОД МОСКВА,5,ОБЛАСТЬ ОРЛОВСКАЯ,5,РЕСПУБЛИКА ТАТАРСТАН,5,ОБЛАСТЬ СМОЛЕНСКАЯ
36608,"Улица (проспект, переулок и т.д.)",6,УЛИЦА ЛЬВА ТОЛСТОГО,7,ШОССЕ КРОМСКОЕ,7,УЛИЦА ДЕМЕНТЬЕВА,7,УЛИЦА ЛАВОЧКИНА
36609,Дом (владение и т.п.),7,16,8,ДОМ 8,8,ДОМ 72 А,8,ДОМ 107А
36621,КПП,22,770401001,20,575201001,25,166101001,19,673201001
36702,Вид,34,УСТАВНЫЙ КАПИТАЛ,32,УСТАВНЫЙ КАПИТАЛ,37,УСТАВНЫЙ КАПИТАЛ,31,УСТАВНЫЙ КАПИТАЛ
